# Laboratory of automatic natural language processing 

## Introduction

This is notebook presenting results from NLP based study. 

THe main aim was creating texts classifier. Firstly, dataset for training was acquired
using webscrapper, then data was processed in order to reject dirty instances. After that,
the features of text was extracted. 

Each feature was word present in dataset, which was counted for each instance and then
the number of occurances of it. It was returned as vector of words.

After that, the classifier using Naive Bayes algorithm was trained and tested on
achieved data.

In [21]:
#Importing modules
from scraperFanFic import *
from sklearn.feature_extraction.text import (TfidfVectorizer,
                                             CountVectorizer)
from nltk.corpus import stopwords
import numpy as np
from nltk import FreqDist
import pandas as pd
from nltk.text import Text
from nltk.text import ContextIndex
from nltk.text import ConcordanceIndex

SITE = "http://fanfiction.net"  # Default site to work on, tested.
from scraperFanFic import *
CATEGORY = 'movie'
UNIVERSE = 'Star-Trek'
scrap = Scrapper(SITE)
print('Scrapper object created!')
scrap.follow_link(CATEGORY)
print('Processing to: ', CATEGORY)
scrap.follow_link(UNIVERSE)
print('Processing to: ', UNIVERSE)
scrap.open(scrap.filtered_fan_fiction())

start = timer()
scrap.preparing_links()
end = timer()
print(end - start)

print("Filtering and preparing links: ", 'DONE')
print("Scrapper will now download the fanfiction into files. \n")
print('=======================================================')
print("Status:")
print("Links on website:", len(scrap.linksOnPage))
print("Links on HDD:", len(scrap.linksOnPage) - len(scrap.checkfiles()), "\n")
print("You have {} links left to scrap.".format(len(scrap.linksToDownload)))
print('=======================================================')

answer = input("Do you want to continue? [Y/N] \n")
if answer.upper() == "Y":

    print('How many links(stories) you want to download? \n ')
    x = input("all or int \n")
    if x.upper() == "ALL":
        story = scrap.download_list(ran_giv=len(scrap.linksToDownload))
    else:
        story = scrap.download_list(ran_giv=int(x))
if answer.upper() == 'N':
    print('Do you want to create a dataset?')
    n = input('[Y/N] \n')
    if n.upper() == "Y":
        story = scrap.read_stories_HDD()
    elif n.upper() == 'N':
        print('Done!')


Scrapper object created!
Processing to:  movie
Processing to:  Star-Trek
0.005135999999993146
Filtering and preparing links:  DONE
Scrapper will now download the fanfiction into files. 

Status:
Links on website: 25
Links on HDD: 0 

You have 25 links left to scrap.
Do you want to continue? [Y/N] 
y
How many links(stories) you want to download? 
 
all or int 
all
Operation completed!


In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io
with open('text.txt','r') as txt:
    text = txt.read()
print('corpus length:', len(text))

Using TensorFlow backend.


corpus length: 6919521


In [2]:
#Pretest
pretest = text[:1000000]
text = pretest

In [3]:

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


total chars: 80


In [4]:

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))


nb sequences: 333320


In [5]:

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1



Vectorization...


In [6]:

# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))


Build model...
Instructions for updating:
Colocations handled automatically by placer.


In [7]:

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()



In [8]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=1,
          callbacks=[print_callback])

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
333320/333320 [==============================] - 249s 747us/step - loss: 1.8512

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "her was everything the evil man had ever"
her was everything the evil man had everyone was to the she would real wont to and they wont to the she would not their command their contrown that he was to their stand with the some on the she would really and they would really work and they wont to their contring to have and they would have to their stand to their faces with the she was to their contrown to their face with the she was to the ship on the ship on the she would have to 
----- diversity: 0.5
----- Generating with seed: "her was everything the evil man had ever"
her was everything the evil man had every can jought want to other to their was nonning a right eaba, and their tonge and from their face contrister of their full want and stand them with so make a p

In [13]:
import pickle
# save the model to disk
# filename = 'finalized_model2.sav'
# pickle.dump(model, open(filename, 'wb'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               107008    
_________________________________________________________________
dense_1 (Dense)              (None, 80)                10320     
Total params: 117,328
Trainable params: 117,328
Non-trainable params: 0
_________________________________________________________________


In [22]:
start_index = random.randint(0, len(text) - maxlen - 1)
for diversity in [0.7]:

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
#     print('----- Generating with seed: "' + sentence + '"')
#     sys.stdout.write(generated)

    for i in range(1000):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

#         sys.stdout.write(next_char)
#         sys.stdout.flush()

In [32]:
# model_pre = pickle.load(open('finalized_model.sav', 'rb'))

neural_texts = list()
fan_fic_len = 400

for n in range(10):
    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.7]:

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence

        for i in range(fan_fic_len):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    neural_texts.append(generated)

In [33]:
random_texts = list()

for n in range(10):
    start_index = random.randint(0, len(text) - maxlen - 1)
    chosen = pretest[start_index:start_index+maxlen+fan_fic_len]
    random_texts.append(chosen)

In [39]:
with open('neuralText.txt','w') as nt:
    for n in range(10):
        nt.write(neural_texts[n] + '\n')

In [40]:
with open('randomText.txt','w') as rt:
    for n in range(10):
        rt.write(random_texts[n] + '\n')